# Step 3: 데이터셋 통합

Dataset 1 (변환된 YOLO)과 Dataset 2를 하나로 통합합니다.

## 왜 통합이 필요한가?
- 두 개의 Kaggle 데이터셋을 하나의 훈련 데이터셋으로 합침
- Dataset 1: helmet만 포함 (4,581개)
- Dataset 2: helmet + vest 포함 (10,500개)
- 통합 후 총 15,081개의 이미지-라벨 쌍

## 파일명 충돌 방지
- 서로 다른 데이터셋에서 같은 파일명이 있을 수 있음
- 예: image_001.jpg가 두 데이터셋 모두에 존재
- 해결: prefix를 추가하여 구분
  - Dataset 1: ds1_image_001.jpg
  - Dataset 2: ds2_image_001.jpg

## 입력/출력
- 입력 1: images/processed/dataset1/ (Step 1 결과)
- 입력 2: images/raw/safety-Helmet-Reflective-Jacket/
- 출력: images/processed/merged/

In [ ]:
import shutil
from pathlib import Path

## 경로 설정

In [ ]:
# 프로젝트 루트 디렉토리
base_dir = Path.cwd().parent

# Dataset 1: Step 1에서 변환된 YOLO 데이터
dataset1_dir = base_dir / 'images' / 'processed' / 'dataset1'

# Dataset 2: 원본 YOLO 데이터
dataset2_dir = base_dir / 'images' / 'raw' / 'safety-Helmet-Reflective-Jacket'

# 출력 디렉토리
output_dir = base_dir / 'images' / 'processed' / 'merged'
output_images_dir = output_dir / 'images'
output_labels_dir = output_dir / 'labels'

print(f"Dataset 1: {dataset1_dir}")
print(f"Dataset 2: {dataset2_dir}")
print(f"출력 경로: {output_dir}")

## 출력 디렉토리 생성

In [ ]:
output_images_dir.mkdir(parents=True, exist_ok=True)
output_labels_dir.mkdir(parents=True, exist_ok=True)
print("✅ 출력 디렉토리 생성 완료")

## Dataset 1 복사 (ds1_ prefix)

In [ ]:
total_images = 0
total_labels = 0

print("📁 Dataset 1 복사 중...")
ds1_images = list((dataset1_dir / 'images').glob('*'))
ds1_labels = list((dataset1_dir / 'labels').glob('*.txt'))

for img in ds1_images:
    new_name = f"ds1_{img.name}"
    shutil.copy(img, output_images_dir / new_name)
    total_images += 1

for lbl in ds1_labels:
    new_name = f"ds1_{lbl.name}"
    shutil.copy(lbl, output_labels_dir / new_name)
    total_labels += 1

print(f"  - 이미지: {len(ds1_images)}개")
print(f"  - 라벨: {len(ds1_labels)}개")

## Dataset 2 복사 (ds2_ prefix)

In [ ]:
print("\n📁 Dataset 2 복사 중...")
ds2_count = {'images': 0, 'labels': 0}

for split in ['train', 'valid', 'test']:
    split_dir = dataset2_dir / split
    if not split_dir.exists():
        continue
    
    images_dir = split_dir / 'images'
    labels_dir = split_dir / 'labels'
    
    if images_dir.exists():
        for img in images_dir.glob('*'):
            if img.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                new_name = f"ds2_{img.name}"
                shutil.copy(img, output_images_dir / new_name)
                ds2_count['images'] += 1
                total_images += 1
    
    if labels_dir.exists():
        for lbl in labels_dir.glob('*.txt'):
            new_name = f"ds2_{lbl.name}"
            shutil.copy(lbl, output_labels_dir / new_name)
            ds2_count['labels'] += 1
            total_labels += 1

print(f"  - 이미지: {ds2_count['images']}개")
print(f"  - 라벨: {ds2_count['labels']}개")

## 결과 확인

In [ ]:
print()
print("=" * 50)
print("✅ Step 3: 통합 완료!")
print("=" * 50)
print(f"   - 총 이미지: {total_images}개")
print(f"   - 총 라벨: {total_labels}개")
print(f"   - 출력 위치: {output_dir}")